## Motivation

In [ ]:
struct Zonotope{N<:Real, VN<:AbstractVector{N}, MN<:AbstractMatrix{N}} <: AbstractZonotope{N}
    center::VN
    generators::MN

    function Zonotope(center::VN, generators::MN) where {N<:Real,
                                                         VN<:AbstractVector{N},
                                                         MN<:AbstractMatrix{N}}
        @assert length(center) == size(generators, 1) "the dimension of the " *
            "center ($(length(center))) and the generators " *
            "($(size(generators, 1))) need to match"
        new{N, VN, MN}(center, generators)
    end
end


## Cassette

In [1]:
using Cassette, Test

Cassette.@context SinToCosCtx

# Override the default recursive `overdub` implementation for `sin(x)`.
# Note that there's no tricks here; this is just a normal Julia method
# overload using the normal multiple dispatch semantics.
Cassette.overdub(::SinToCosCtx, ::typeof(sin), x) = cos(x)

x = rand(10)
y = Cassette.overdub(SinToCosCtx(), sum, i -> cos(i) + sin(i), x)
@test y == sum(i -> 2 * cos(i), x)

Test Passed

In [2]:
using Revise, LazySets, Cassette, InteractiveUtils, Test

In [3]:
@inferred Zonotope(rand(2), hcat(rand(2, 2), zeros(2, 3)))

ErrorException: return type Zonotope{Float64,Array{Float64,1},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false}} does not match inferred return type Zonotope{Float64,Array{Float64,1},_A} where _A<:AbstractArray{Float64,2}

In [4]:
Zonotope(rand(2), hcat(rand(2, 2), zeros(2, 3)))

Zonotope{Float64,Array{Float64,1},SubArray{Float64,2,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Array{Int64,1}},false}}([0.7560355513851835, 0.18694069197769791], [0.1765580694882234 0.32682022455753645; 0.12875757743660876 0.20503548958114814])

In [5]:
using LazySets, Cassette

Cassette.@context ZonotopeCtx
#const zonotope_ctx = Cassette.disablehooks(ZonotopeCtx())

function Cassette.overdub(::ZonotopeCtx, ::typeof(Zonotope), center::VN, generators::MN) where {N, VN<:AbstractVector{N}, MN<:AbstractMatrix{N}}
    # disable generators removal
    Zonotope(center, generators, remove_zero_generators=false)
end

In [6]:
Cassette.overdub(ZonotopeCtx(), Zonotope, rand(2), hcat(rand(2, 2), zeros(2, 3)))

Zonotope{Float64,Array{Float64,1},Array{Float64,2}}([0.24274789256436424, 0.45282421956931196], [0.9340341497332454 0.6573075187868196 … 0.0 0.0; 0.2519076269549425 0.2625423719784101 … 0.0 0.0])

In [9]:
zonotope(f) = Cassette.overdub(ZonotopeCtx(), f)

zonotope (generic function with 1 method)

In [10]:
zonotope() do
    Z = Zonotope(rand(2), hcat(rand(2, 2), zeros(2, 3)))
    Z.generators
end

2×5 Array{Float64,2}:
 0.836691  0.968074  0.0  0.0  0.0
 0.387156  0.219346  0.0  0.0  0.0

In [14]:
using ReachabilityAnalysis

include("/home/mforets/.julia/dev/ReachabilityAnalysis/test/models/linear/motor.jl")

prob, tspan = motor_homog()
sol = solve(prob, tspan=tspan, GLGM06(δ=0.01), static=true);

In [12]:
zonotope() do
    sol = solve(prob, tspan=tspan, GLGM06(δ=0.01), static=true);
end

MethodError: MethodError: Cannot `convert` an object of type 
  Zonotope{Float64{},StaticArrays.SArray{Tuple{8},Float64{},1,8},SubArray{Float64,2,StaticArrays.SArray{Tuple{8,8},Float64,2,64},Tuple{Base.Slice{StaticArrays.SOneTo{8}},Array{Int64,1}},false}} to an object of type 
  Zonotope{Float64{},StaticArrays.SArray{Tuple{8},Float64{},1,8},StaticArrays.SArray{Tuple{8,8},Float64,2,64}}
Closest candidates are:
  convert(::Type{#s16} where #s16<:Union{Number, T}, !Matched::MultivariatePolynomials.AbstractPolynomialLike{T}) where T at /home/mforets/.julia/packages/MultivariatePolynomials/Mpv8c/src/conversion.jl:16
  convert(::Type{T}, !Matched::T) where T at essentials.jl:171

In [ ]:
Cassette.overdub(ZonotopeCtx(), Zonotope, rand(2), hcat(rand(2, 2), zeros(2, 3)))

In [ ]:
using Cassette, IntervalArithmetic

Cassette.@context FastPowersCtx
const fast_powers_ctx = Cassette.disablehooks(FastPowersCtx())

Cassette.overdub(::FastPowersCtx, ::typeof(^), a, b) = pow(a,b)

fast_powers(f) = Cassette.overdub(fast_powers_ctx, f)

x = 1.1..2.2



In [ ]:
fast_powers() do
    x^10
end

## IRTools

In [ ]:
using Revise, LazySets, IRTools

In [ ]:
f(x) = x+x

In [ ]:
@code_ir f(1)

In [ ]:
@code_ir Zonotope([0.0, 1.0], [0.0 1.0; 0.0 1.0])

In [ ]:
@code_ir Zonotope(rand(2), hcat(rand(2, 2), zeros(2, 3)))